In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image

import scipy.misc

import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
import dense_correspondence.correspondence_tools.correspondence_finder as correspondence_finder
from dense_correspondence.correspondence_tools.correspondence_finder import random_sample_from_masked_image, pinhole_projection_image_to_world
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType
from dense_correspondence.network.dense_correspondence_network import DenseCorrespondenceNetwork
import dense_correspondence.evaluation.plotting as dc_plotting
%matplotlib inline 
utils.set_cuda_visible_devices([0])

In [ ]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
config = utils.getDictFromYamlFilename(config_filename)
dce = DenseCorrespondenceEvaluation(config)
DCE = DenseCorrespondenceEvaluation
# dcn = dce.load_network_from_config("cup_bottlesauce_elephant_cat_pear_unit_16") # this worked pretty well
dcn = dce.load_network_from_config("thingiverse_multi_rgb_unit_16") 

dataset = dcn.load_training_dataset()
camera_intrinsics = dataset.get_camera_intrinsics("elephant")

In [ ]:
def run_dense_descriptor_original_match(dcn, dataset, heap_name, target_name, num_matches=5, patch_size=None, image_type="depth"):
    """Use the original dense descriptor sampling + matching method"""
    
    if image_type == "rgb":
        target_img = Image.open("images/targets/{}/target_rgb.png".format(target_name)).convert('RGB')
    else:
        target_img = Image.open("images/targets/{}/target_depth.png".format(target_name)).convert('RGB')
    target = np.array(target_img)
    target_descriptors = run_dcn_on_image(dcn, dataset, target_img)
    target_mask = np.load("imageseshre`/targets/{}/target_mask.npy".format(target_name)).astype(np.bool)
    
    if image_type == "rgb":
        heap_img_rgb = Image.open("images/heaps/{}/heap_rgb.png".format(heap_name)).convert('RGB')
    else:
        heap_img = Image.open("images/heaps/{}/heap_depth.png".format(heap_name)).convert('RGB')    
    heap = np.array(heap_img)
    heap_descriptors = run_dcn_on_image(dcn, dataset, heap_img)
    heap_mask = np.load("images/heaps/{}/heap_mask.npy".format(heap_name)).astype(np.bool)
    
    DCE.single_image_pair_qualitative_analysis(dcn, dataset, target, heap, target_mask, heap_mask, num_matches, patch_size)


In [ ]:
# run_dense_descriptor_original_match(dcn, dataset, )
heap_num = 5
dirname = "/home/davidtseng/pytorch-dense-correspondence/dense_correspondence/experiments/mech_search/images/heatmap/10_objs_1_0_ratio_perturbed_80deg"
# 80 deg
target_img = Image.open(dirname + "/{}/target/rgb.png".format(heap_num)).convert('RGB')
target = np.array(target_img)
target_mask = np.load(dirname + "/{}/target/mask.npy".format(heap_num)).astype(np.bool)

    
heap_img = Image.open(dirname + "/{}/heap/rgb.png".format(heap_num)).convert('RGB')
heap = np.array(heap_img)
heap_mask = np.load(dirname + "/{}/heap/mask.npy".format(heap_num)).astype(np.bool)


DCE.single_image_pair_qualitative_analysis(dcn, dataset, target, heap, target_mask, heap_mask, num_matches=20, patch_size=None)
